In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

import ray
ray.init(num_cpus=10)

2022-11-02 11:09:25,316	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 


Python version:,3.10.4
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8267


In [2]:
# Create Tensors to hold input and outputs.
#1,3,1+1,2,1
beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0
prior=[1,2,3,4,5]

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [3]:
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)



    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs_1):
        lambda1,lr,ga= config["lambda1"],config["lr"],config["ga"]
        model_1=MLP(seed=1)
        model_2=MLP(seed=1)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1= model_1(inputs_1[idx_train])
            out_2= model_2(inputs_2[idx_train])
            out_3= model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            #[:-1] for leaving out bias term#
            model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])
           

            model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])
    

            model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])
    


            # compute loss
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            
            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

#             l2_regularization = lambda2* torch.sum(all_linear23_params.pow(2))
            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
#             scheduler.step()
        
        
        test_out_1= model_1(train_x1[idx_test])
        test_out_2= model_2(train_x2[idx_test])
        test_out_3= model_3(train_x3[idx_test])
        
        test_BCE_loss_1 = loss_fn(test_out_1, targets_1[idx_test].reshape(-1,1))
        test_BCE_loss_2 = loss_fn(test_out_2, targets_2[idx_test].reshape(-1,1))
        test_BCE_loss_3 = loss_fn(test_out_3, targets_3[idx_test].reshape(-1,1))
        
        test_BCE_loss=test_BCE_loss_1+test_BCE_loss_2+test_BCE_loss_3
        
        sum_test_BCE_loss+=test_BCE_loss.item()

    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)
     

In [4]:
def train_model_2(config):
    split_num=5
    kf = KFold(n_splits=split_num)
    
    
    lambda1,lr,eta= config["lambda1"], config["lr"],config["eta"]
    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3
    
    
    
    sum_tGM=0
    for idx_train,idx_test in kf.split(inputs_1):
       
        
        model_1=MLP(seed=1)
        model_2=MLP(seed=1)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1 = model_1(inputs_1[idx_train])
            out_2 = model_2(inputs_2[idx_train])
            out_3 = model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])
           

            model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])
    

            model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])
    


            # compute loss

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            
            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))
            
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out_1= model_1(train_x1[idx_test])
        test_out_2= model_2(train_x2[idx_test])
        test_out_3= model_3(train_x3[idx_test])
        
        test_prediction_1= (model_1(train_x1[idx_test])>0.5).clone().int()
        test_target_1= train_y1[idx_test].reshape(-1,1).int()

        test_prediction_2= (model_2(train_x2[idx_test])>0.5).clone().int()
        test_target_2= train_y2[idx_test].reshape(-1,1).int()

        test_prediction_3= (model_3(train_x3[idx_test])>0.5).clone().int()
        test_target_3= train_y3[idx_test].reshape(-1,1).int()

        test_prediction=np.append(test_prediction_1,test_prediction_2)
        test_prediction=np.append(test_prediction,test_prediction_3)
        test_prediction=test_prediction.tolist()

        test_target=np.append(test_target_1,test_target_2)
        test_target=np.append(test_target,test_target_3)
        test_target=test_target.tolist()

        final_matrix=confusion_matrix(test_target,test_prediction)
        tTPR=recall_score(test_target,test_target)#TPR
        tTNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
        tGM=math.sqrt(tTPR*tTNR)

        sum_tGM+=tGM

    tune.report(my_test_sum_tGM=sum_tGM/split_num)
     

In [5]:
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

(raylet) [2022-11-02 11:09:35,308 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 588627968; capacity: 29394726912. Object creation will fail if spilling is required.


In [6]:
for i in range(10):
    setup_seed(i)
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)

    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
################## first step######################################  
    prior=[1,2,3,4,5]

# choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1= torch.nn.Linear(p,1)

        def forward(self, x):
            out = torch.sigmoid(self.linear1(x))
            return out

    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "ga":tune.choice([0.3,0.4,0.5])
        }

    result = tune.run(
            train_model,  
            config=config,
            num_samples=30)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lr,ga=final_lambda["lambda1"],final_lambda["lr"],final_lambda["ga"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]
    
    model_1=MLP(seed=1)
    model_2=MLP(seed=1)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

    loss_record=[]
    
    for t in range(max_iteration):
    
    # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1= model_1(inputs_1)
        out_2= model_2(inputs_2)
        out_3= model_3(inputs_3)



        model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])


        model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])


        model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])


        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)

        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))




    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))
        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization 

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()

    
################## Y_prior##################        
    y_prior_1= (model_1(train_x1)).detach().squeeze()

    y_prior_2= (model_2(train_x2)).detach().squeeze()

    y_prior_3= (model_3(train_x3)).detach().squeeze()
    
    
################## third step##################

    #no prior
    prior=[]
    # choose lambda1,lambda2 by ray tune   
        
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1= torch.nn.Linear(p,1)

        def forward(self, x):
            out = torch.sigmoid(self.linear1(x))
            return out
        
        
    config={
        "lambda1": tune.choice([0.03,0.04,0.05]),
         "eta": tune.choice([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]),
         "lr":tune.choice([0.6,0.7,0.8])
                            }

    result = tune.run(
            train_model_2,  
            config=config,
            num_samples=100)
    

################## fourth step##################

    #no prior
    prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    lambda1,eta,lr=final_parameter["lambda1"],final_parameter["eta"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000

    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3

    # record loss descent
    BCE=[]


    # main nn object
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1= torch.nn.Linear(p,1)

        def forward(self, x):
            out = torch.sigmoid(self.linear1(x))
            return out



    model_1=MLP(seed=1)
    model_2=MLP(seed=1)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

    # learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
    # loop for max_iteration times
    
    for t in range(max_iteration):
    
    # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1= model_1(inputs_1)
        out_2= model_2(inputs_2)
        out_3= model_3(inputs_3)



        # extract parameters
        #[:-1] for leaving out bias term#

        model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])


        model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])


        model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])

        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))
        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
    
####################################   prediction ####################################      

    prediction_1= (model_1(test_x1)>0.5).clone().int()
    target_1= test_y1.reshape(-1,1).int()

    prediction_2= (model_2(test_x2)>0.5).clone().int()
    target_2= test_y2.reshape(-1,1).int()

    prediction_3= (model_3(test_x3)>0.5).clone().int()
    target_3= test_y3.reshape(-1,1).int()

    prediction=np.append(prediction_1,prediction_2)
    prediction=np.append(prediction,prediction_3)
    prediction=prediction.tolist()

    target=np.append(target_1,target_2)
    target=np.append(target,target_3)
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))
    
    
####################################   variable selection ####################################     
    model_1_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_2_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_3_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.12
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=cubic_root(model_1_weight[0,j]*model_2_weight[0,j]*model_3_weight[0,j])

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_1_weight[0,j]=0
            model_2_weight[0,j]=0
            model_3_weight[0,j]=0
        else:
            model_1_weight[0,j]=model_1_weight[0,j]
            model_2_weight[0,j]=model_2_weight[0,j]
            model_3_weight[0,j]=model_3_weight[0,j]
    
    index=np.nonzero(model_1_weight)[1]
    
    vs[:,index]+=1
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    TP=len(trueindex.intersection(index))
    FP=len(np.setdiff1d(index,trueindex))
    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    
    print(i)
    

2022-11-02 11:09:42,047	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lr,iter,total time (s),my_test_BCE_loss
train_model_cb797_00000,TERMINATED,10.224.0.12:2206,0.5,0.03,0.8,1,5.2305,0.595857
train_model_cb797_00001,TERMINATED,10.224.0.12:2232,0.3,0.05,0.6,1,4.79477,0.670307
train_model_cb797_00002,TERMINATED,10.224.0.12:2234,0.3,0.05,0.6,1,4.79097,0.670307
train_model_cb797_00003,TERMINATED,10.224.0.12:2236,0.4,0.03,0.8,1,4.78056,0.595857
train_model_cb797_00004,TERMINATED,10.224.0.12:2238,0.4,0.05,0.8,1,4.82607,0.676702
train_model_cb797_00005,TERMINATED,10.224.0.12:2240,0.4,0.04,0.7,1,4.81136,0.62506
train_model_cb797_00006,TERMINATED,10.224.0.12:2242,0.5,0.04,0.7,1,4.83399,0.62506
train_model_cb797_00007,TERMINATED,10.224.0.12:2243,0.4,0.04,0.6,1,4.82222,0.6269
train_model_cb797_00008,TERMINATED,10.224.0.12:2246,0.4,0.04,0.7,1,4.72951,0.62506
train_model_cb797_00009,TERMINATED,10.224.0.12:2248,0.3,0.04,0.7,1,4.73909,0.62506


(raylet) [2022-11-02 11:09:45,316 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 588185600; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_cb797_00000:
  date: 2022-11-02_11-09-49
  done: false
  experiment_id: 78ee6ba3cfbd4d9b863907500951ef29
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.595856636762619
  node_ip: 10.224.0.12
  pid: 2206
  time_since_restore: 5.230504989624023
  time_this_iter_s: 5.230504989624023
  time_total_s: 5.230504989624023
  timestamp: 1667358589
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: cb797_00000
  warmup_time: 0.0037910938262939453
  
Result for train_model_cb797_00000:
  date: 2022-11-02_11-09-49
  done: true
  experiment_id: 78ee6ba3cfbd4d9b863907500951ef29
  experiment_tag: 0_ga=0.5000,lambda1=0.0300,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.595856636762619
  node_ip: 10.224.0.12
  pid: 2206
  time_since_restore: 5.230504989624023
  time_this_iter_s: 5.230504989624023
  time_total_s: 5.230504989624023
  timestamp: 1667358589
  timesteps_since_restore: 0
  

Result for train_model_cb797_00005:
  date: 2022-11-02_11-09-51
  done: true
  experiment_id: 411daff2428041e199228e6c162dc953
  experiment_tag: 5_ga=0.4000,lambda1=0.0400,lr=0.7000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6250597536563873
  node_ip: 10.224.0.12
  pid: 2240
  time_since_restore: 4.811360120773315
  time_this_iter_s: 4.811360120773315
  time_total_s: 4.811360120773315
  timestamp: 1667358591
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: cb797_00005
  warmup_time: 0.004476070404052734
  
Result for train_model_cb797_00010:
  date: 2022-11-02_11-09-54
  done: false
  experiment_id: 78ee6ba3cfbd4d9b863907500951ef29
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6268995881080628
  node_ip: 10.224.0.12
  pid: 2206
  time_since_restore: 5.649412155151367
  time_this_iter_s: 5.649412155151367
  time_total_s: 5.649412155151367
  timestamp: 1667358594
  timesteps_since_restore: 0
 

...

(raylet) [2022-11-02 13:28:12,830 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 449970176; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:28:22,840 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 449949696; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:28:32,849 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 449900544; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:28:42,859 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 449863680; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-02 13:33:23,128 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 448950272; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:33:33,137 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 448905216; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:33:43,147 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 448876544; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:33:53,156 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 448843776; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-02 13:38:33,424 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 447901696; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:38:43,434 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 447877120; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:38:53,444 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 447832064; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:39:03,453 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 447803392; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-02 13:43:43,720 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 446840832; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:43:53,729 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 446824448; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:44:03,739 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 446808064; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:44:13,748 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 446767104; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-02 13:48:54,018 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 445784064; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:49:04,027 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 445767680; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:49:14,037 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 445718528; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 13:49:24,047 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 445689856; capacity: 29394726912. Object creation will fail if spilling i

In [7]:
####################################   prediction ####################################   
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)


0.9230139404045874

0.9323278100995062

0.927320541971528

0.9275

0.9322587941946752

0.9269635813866909

(raylet) [2022-11-02 14:09:55,234 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 441106432; capacity: 29394726912. Object creation will fail if spilling is required.


In [8]:
####################################   variable selection ####################################   
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

1.0

0.8866666666666667

0.9416268242330533

0.10200000000000001

0.898

In [9]:
vs

array([[10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [10]:
i

9

(raylet) [2022-11-02 14:10:05,243 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 441094144; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:10:15,252 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 441049088; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:10:25,262 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 441028608; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:10:35,271 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 440979456; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-02 14:15:15,532 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 436482048; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:15:25,535 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 435785728; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:15:35,540 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 435032064; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:15:45,545 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 434286592; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-02 14:20:25,743 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 412254208; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:20:35,750 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 411435008; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:20:45,754 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 410705920; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:20:55,759 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 409968640; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-02 14:25:35,912 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 388149248; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:25:45,916 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 387411968; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:25:55,921 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 386670592; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:26:05,927 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 385867776; capacity: 29394726912. Object creation will fail if spilling i

...

(raylet) [2022-11-02 16:50:22,580 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 19828736; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:50:32,589 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 19771392; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:50:42,597 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 19730432; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:50:52,606 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 19611648; capacity: 29394726912. Object creation will fail if spilling is re

(raylet) [2022-11-02 16:55:32,856 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 9461760; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:55:42,874 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 7868416; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:55:52,891 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 6766592; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:56:02,905 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 5705728; capacity: 29394726912. Object creation will fail if spilling is requir

(raylet) [2022-11-02 17:00:53,155 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 52989952; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 17:01:03,164 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 52924416; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 17:01:13,172 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 52805632; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 17:01:23,180 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 52559872; capacity: 29394726912. Object creation will fail if spilling is re

(raylet) [2022-11-02 17:06:03,430 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 37949440; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 17:06:13,439 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 36265984; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 17:06:23,447 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 35217408; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 17:06:33,458 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 33878016; capacity: 29394726912. Object creation will fail if spilling is re

(raylet) [2022-11-02 17:11:13,686 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 10670080; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 17:11:23,694 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 10567680; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 17:11:33,703 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 10457088; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 17:11:43,711 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 10342400; capacity: 29394726912. Object creation will fail if spilling is re